<a href="https://colab.research.google.com/github/ramjv/CNN-Model-optimize-using-Keras-Tuner/blob/master/fmnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Conv2D,Flatten,MaxPooling2D
from tensorflow.keras.utils import to_categorical

In [6]:
print(keras.__version__)
print(tf.__version__)

2.3.0-tf
2.2.0-rc2


In [7]:
batch_size = 128
num_classes = 10
epochs = 12

(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

img_rows, img_cols = 28, 28
train_images = train_images.reshape(len(train_images),img_rows,img_cols,1)
test_images = test_images.reshape(len(test_images),img_rows,img_cols,1)
input_shape = (img_rows, img_cols, 1)
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images /= 255
test_images /= 255

print('x_train shape:', train_images.shape)
print(train_images.shape[0], 'train samples')
print(test_images.shape[0], 'test samples')

train_labels = to_categorical(train_labels,num_classes)
test_labels = to_categorical(test_labels,num_classes)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [8]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64,kernel_size=(3 ,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(train_images, train_labels,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(test_images, test_labels))
score = model.evaluate(test_images, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/12
469/469 [==============================] - 5s 11ms/step - loss: 2.2779 - accuracy: 0.1550 - val_loss: 2.2345 - val_accuracy: 0.2862
Epoch 2/12
469/469 [==============================] - 5s 10ms/step - loss: 2.2050 - accuracy: 0.2700 - val_loss: 2.1457 - val_accuracy: 0.5348
Epoch 3/12
469/469 [==============================] - 5s 10ms/step - loss: 2.1110 - accuracy: 0.3704 - val_loss: 2.0297 - val_accuracy: 0.5773
Epoch 4/12
469/469 [==============================] - 5s 10ms/step - loss: 1.9882 - accuracy: 0.4320 - val_loss: 1.8723 - val_accuracy: 0.5919
Epoch 5/12
469/469 [==============================] - 5s 10ms/step - loss: 1.8272 - accuracy: 0.4738 - val_loss: 1.6776 - val_accuracy: 0.5983
Epoch 6/12
469/469 [==============================] - 5s 10ms/step - loss: 1.6580 - accuracy: 0.4967 - val_loss: 1.4841 - val_accuracy: 0.6012
Epoch 7/12
469/469 [==============================] - 5s 10ms/step - loss: 1.5097 - accuracy: 0.5146 - val_loss: 1.3274 - val_accuracy: 0.6107